In [0]:
%pip install python-dotenv

In [0]:
from dotenv import load_dotenv
from pyspark.sql.functions import *
import os
import requests
import time
from datetime import datetime
from bs4 import BeautifulSoup
from pyspark.sql.types import StructType, StructField, StringType, DateType

In [0]:
catalog_dev = "`land_topografisk-gdb_dev`"
schema_dev = "ai2025"
spark.sql(f"USE CATALOG {catalog_dev}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_dev}")
spark.sql(f"USE SCHEMA {schema_dev}")
bronze_table = "hospitals_bronze"
silver_table = "hospitals_silver"

BASE_PATH = "/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER"

In [0]:
q = f"""
CREATE TABLE IF NOT EXISTS {silver_table} (
    lokalid STRING,
    geometry BINARY,
    bbox ARRAY<DOUBLE>,
    bbox_str STRING,
    image_wms STRING,
    image_status STRING,
    image_path STRING,
    kommunenummer BIGINT,
    oppdateringsdato TIMESTAMP,
    ingest_time TIMESTAMP,
    load_time TIMESTAMP,
    photo_time TIMESTAMP
) USING DELTA
"""
spark.sql(q)

In [0]:
def read_table_to_wkt():
    """
    Leser polygonene fra bronsetabellen og returnerer dem som en GeoDataFrame med WKT geometri.
    """
    df_bronze = spark.read.table(bronze_table).withColumn(
        "geometry", expr("ST_GeomFromWKB(geometry)")
    )
    df = (
        df_bronze
        .withColumn("_geom_srid", expr("ST_SetSRID(geometry, 4326)"))
        .withColumn("geometry", expr("ST_Transform(_geom_srid, 'EPSG:25833')"))
        .select("lokalid", "geometry", "kommunenummer", "oppdateringsdato", "ingest_time")
    )
    return df

In [0]:
def make_envelope(df: DataFrame) -> DataFrame:
    """
    Lager en minimal boks rundt polygonene.
    """
    return df.withColumn(
        "envelope",
        expr(
            """
                CASE 
                    WHEN ST_GeometryType(geometry) = 'ST_Point' 
                        THEN ST_Envelope(ST_Buffer(geometry, 128))
                    ELSE ST_Envelope(geometry)
                END
                """
        ),
    )

In [0]:
def envelope_to_bboxes(df: DataFrame, tile_width=512, tile_height=512) -> DataFrame:
    """
    Tar inn envelope og returnerer bboxer med en gitt størrelse
    """
    df = (
        df.withColumn("xmin", expr("ST_XMin(envelope)"))
        .withColumn("ymin", expr("ST_YMin(envelope)"))
        .withColumn("xmax", expr("ST_XMax(envelope)"))
        .withColumn("ymax", expr("ST_YMax(envelope)"))
    )

    df = df.withColumn(
        "n_x",
        greatest(
            lit(1),
            ceil((col("xmax") - col("xmin")) / lit(float(tile_width))).cast("int"),
        ),
    ).withColumn(
        "n_y",
        greatest(
            lit(1),
            ceil((col("ymax") - col("ymin")) / lit(float(tile_height))).cast("int"),
        ),
    )

    df = df.withColumn("x_idx_arr", sequence(lit(0), col("n_x") - 1)).withColumn(
        "y_idx_arr", sequence(lit(0), col("n_y") - 1)
    )

    df = df.withColumn("x_idx", explode("x_idx_arr")).withColumn(
        "y_idx", explode("y_idx_arr")
    )

    df = df.withColumn(
        "x", col("xmin") + col("x_idx") * lit(float(tile_width))
    ).withColumn("y", col("ymin") + col("y_idx") * lit(float(tile_height)))

    df = df.withColumn(
        "tile",
        expr(
            "ST_PolygonFromEnvelope(x, y, x + {0}, y + {1})".format(
                tile_width, tile_height
            )
        ),
    ).withColumn(
        "bbox",
        expr(f"array(x, y, x + {tile_width}, y + {tile_height})"))
    df = df.withColumn("bbox_str", concat_ws(",", col("bbox")))
    
    drop_cols = [
        "xmin",
        "ymin",
        "xmax",
        "ymax",
        "n_x",
        "n_y",
        "x_idx_arr",
        "y_idx_arr",
        "x_idx",
        "y_idx",
        "envelope",
        "x",
        "y",
        "tile",
    ]
    for c in drop_cols:
        if c in df.columns:
            df = df.drop(c)

    return df

In [0]:
def generate_image_url(bbox_str: str) -> str:
    """
    Genererer en URL for image-bilde basert på bbox_str.
    """
    width, height = [512, 512]
    return (
        f"https://wms.geonorge.no/skwms1/wms.nib?VERSION=1.3.0"
        f"&service=WMS&request=GetMap&Format=image/png&"
        f"GetFeatureInfo=text/plain&CRS=EPSG:25833&Layers=ortofoto&"
        f"BBox={bbox_str}&width={width}&height={height}"
    )


def image_file_exists(lokalid: str) -> str:
    """
    Sjekker om ortofoto-filen eksisterer for en gitt lokalid.
    """
    path = f"/Volumes/land_topografisk-gdb_dev/external_dev/static_data/DL_SNUPLASSER/hospital_images/image_{lokalid}.png"
    return "DOWNLOADED" if os.path.exists(path) else "PENDING"

In [0]:
load_dotenv()
BRUKERID = os.getenv("GEONORGE_BRUKERID")
PASSORD = os.getenv("GEONORGE_PASSORD")


def get_token():
    """
    Hender token fra GeoNorge og returnerer det.
    """
    url = (
        f"https://baat.geonorge.no/skbaatts/req?brukerid={BRUKERID}"
        f"&passord={PASSORD}&tjenesteid=wms.nib&retformat=s"
    )
    raw_token = requests.get(url).text.strip("`")
    return raw_token


token = get_token()
token_start_time = time.time()
token_lifetime = 55 * 60


def refresh_token_if_needed():
    """
    henter ny token om den gamle er utløpt.
    """
    global token, token_start_time
    if time.time() - token_start_time > token_lifetime:
        print("🔄 Fornyer token...")
        token = get_token()
        token_start_time = time.time()

In [0]:
def get_fotodato(bbox: str, token: str, max_retries=10):
    """
    Henter fotodato for en bbox.
    """
    url = f"https://wms.geonorge.no/skwms1/wms.nib?SERVICE=WMS&VERSION=1.3.0&REQUEST=GetFeatureInfo&CRS=EPSG:25833&BBOX={bbox}&WIDTH=512&HEIGHT=512&LAYERS=ortofoto&QUERY_LAYERS=ortofoto&INFO_FORMAT=text/html&I=256&J=256&TICKET={token}"

    table = None
    field_value = None
    for i in range(max_retries):
        try:
            response = requests.get(url, timeout=10)
            time.sleep(1.0)
            soup = BeautifulSoup(response.text, "html.parser")

            table = soup.find("table")
            break
        except Exception as e:
            wait = 2**i
            print(f"⚠️ Feil ved henting av fotodato ({e}), prøver igjen om {wait}s...")
            time.sleep(wait)

    if table:
        rows = table.find_all("tr")
        for row in rows:
            cells = row.find_all("td")
            if len(cells) >= 2 and cells[0].text.strip() == "Fotodato":
                field_value = cells[1].text.strip()
                field_value = datetime.strptime(field_value, "%d.%m.%Y").date()
                return field_value

    return None

In [0]:
def add_ortofoto_date(df: DataFrame, token: str) -> DataFrame:
    """
    Legger til fotodato kolonnen til en DataFrame.
    """
    # Henter bare relevante kolonner
    sample_rows = df.select("lokalid", "bbox_str").collect()

    # Henter fotodato
    bbox_date_pairs = [
        (row["lokalid"], get_fotodato(row["bbox_str"].replace("_", ","), token))
        for row in sample_rows
    ]

    schema = StructType(
        [
            StructField("lokalid", StringType(), True),
            StructField("photo_time", DateType(), True),
        ]
    )

    bbox_date_df = spark.createDataFrame(bbox_date_pairs, schema)
    df_with_date = df.join(bbox_date_df, on="lokalid", how="left")

    return df_with_date

In [0]:
def write_delta_table(sdf: DataFrame, mode: str = "merge") -> None:
    """
    Skriver data til deltatabellen og opdaterer dersom lokalid allerede finnes.
    """
    if mode == "overwrite":
        sdf.write.format("delta").option("mergeSchema", "true").mode(
            "overwrite"
        ).saveAsTable(silver_table)
    else:
        from delta.tables import DeltaTable

        delta_tbl = DeltaTable.forName(spark, silver_table)

        delta_tbl.alias("target").merge(
            sdf.alias("source"), condition="target.lokalid = source.lokalid"
        ).whenMatchedUpdate(
            condition="target.load_time < source.load_time OR target.image_path IS NULL",
            set={col: f"source.{col}" for col in sdf.columns},
        ).whenNotMatchedInsert(
            values={col: f"source.{col}" for col in sdf.columns}
        ).execute()

In [0]:
def main():
    df = read_table_to_wkt()
    df = make_envelope(df)
    df = envelope_to_bboxes(df)
    generate_image_url_udf = udf(generate_image_url, StringType())
    image_file_exists_udf = udf(image_file_exists, StringType())
    df = (
        df.withColumn("image_status", image_file_exists_udf("lokalid"))
        .withColumn(
            "image_path",
            concat(
                lit(BASE_PATH),
                lit("/hospital_images/image_"),
                col("lokalid"),
                lit(".png"),
            ),
        )
        .withColumn("image_wms", generate_image_url_udf("bbox_str"))
    )
    df = df.withColumn("load_time", current_timestamp())
    df = add_ortofoto_date(df, token)
    df = df.withColumn("geometry", expr("ST_AsBinary(geometry)"))
    
    write_delta_table(df)

In [0]:
main()